# <center><ins> Coastal flood hazard mapping (using bathtub method) </ins></center>

This notebook aims to describe step-by-step the methodology followed for the creation of flooding maps using (1) a DTM (2) and a shapefile with the polygons of areas of influence.
 In this example, the flooding level is calculated by 100y return period for the **Total Water Level (TWL)**, understood as:
 
$$ TWL = SWL + Ru $$
where:
<center> SWL: Still Water Level (astronomical tide + storm surge) </center>
<center> Ru: Wave runup </center>

<center><img src="images/TWL_scheme.png" width="400"></center>
<center><i> Total Water Level scheme </i></center>

### TWL calculation methodology
The **Ru component** of the TWL have been **calculated by sempi-empirical formulae** having previously segmented the coastline into 417 stretches. According to the geomorphology type of each strech, it has been considered the following equations:
<br/><br/>
<center><img src="images/table-formulae.png" width="500"></center>

<i> Equation 5.1 in EurOtop, 2018 for relatively gentle slopes in EurOtop, 2018. Manual on wave overtopping of sea defences and related structures. Van der Meer, J.W., Allsop, N.W.H., et al. www.overtopping-manual.com

<i> Stockdon, H. F., Holman, R. A., Howd, P. A., & Sallenger Jr, A. H. (2006). Empirical parameterization of setup, swash, and runup. Coastal engineering, 53(7), 573-588. </i>


### Flood maps generation methodology
The **methodology** selected for **flood mapping** is the **"bathtub approach"**, which is a common and simple method for carrying out flood analysis in the coastal zone. The method basically consists of taking all pixels in a given area below a TWL value and considering them as flooded.

<div class="alert alert-warning">
  <strong>This notebook can only be run in the ArcGIS Pro Python window. Any other Python IDE will not work.</strong>
</div>
<br/>
<center><img src="images/python-window-arcgis-pro.png" width="300"></center>

### Load the required libraries and set the local work environment

In [5]:
import arcpy
from arcpy.sa import *
import os

In [2]:
# File geodatabase
workspace = r'D:\Compartida_CCover\Clima\Total Water Level (TWL)\GIS_outputs\TWL.gdb'

# DTM raster file that will be croped
rasterDTM = r'D:\Compartida_CCover\Stakeholder_Data\BN_Baseline\BN2_PA_SIntegraM_DTM\DTM_SIntegraM_Tiles_2kmbuffer\DTM_all_tiles.tif' 

# Polygon Shapefile with Coastal Stretches segmentation
maskShape = r'D:\Compartida_CCover\Clima\Total Water Level (TWL)\GIS_outputs\TWL.gdb\CoastalStretch_pol'

# Relevant field names within the CS polygon shapefile
maskField = u'CS_id'
CU = u'CU_id'
TWLField = u'TWL_100y_Tr'

In [3]:
# Set work environment
arcpy.env.workspace = workspace
arcpy.env.scratchWorkspace = workspace
arcpy.env.overwriteOutput = True

aprx = arcpy.mp.ArcGISProject("CURRENT")
#aprxMap = aprx.listMaps("Map")[0]

### Do the operations within a for loop 

In [ ]:
for row in arcpy.da.SearchCursor(maskShape, ['SHAPE@', maskField, TWLField, CU]):
    
    # Create a name with the stretch ID
    stretchName = "CS_id_{}".format(row[1])
       
    # Create Feature Layer of single CoastalStretch_ID
    arcpy.analysis.Select(maskShape, "{}\\{}".format(workspace,"tempFeature"), stretchName)
    
    # Run Extract By Mask
    extractOut = ExtractByMask(rasterDTM, "tempFeature")
    extractOut.save("{}\\{}".format(workspace, "tempMDTRaster"))
    
    # Put ones for the flooded values and zeros for the non-flooded values and save the raster
    reclassOut = arcpy.sa.Reclassify(extractOut, "VALUE",
                                     "-9999 0 NODATA;0 "+ str(row[2]) +" 1;"+ str(row[2]) +" 9999 NODATA",
                                     "DATA")
    reclassOut.save("{}\\{}".format(workspace, "tempReclassRaster"))
    
    # Convert the raster into a polygon shapefile
    TWLpol = arcpy.conversion.RasterToPolygon(reclassOut, "tempRas2Pol", "SIMPLIFY", "Value",
                                              "SINGLE_OUTER_PART", None)
    
    # Dissolve the features of each shape into one single feature
    TWLpoldiss = arcpy.management.Dissolve(TWLpol, "{}\\{}".format(workspace, "TWL_"+stretchName),
                                           None, None, "MULTI_PART", "DISSOLVE_LINES")
    
    # Add 3 fields to the new shapefile: CU_id, CS_id and TWL
    arcpy.AddField_management(TWLpoldiss,"CU_id","TEXT", field_length=10)
    arcpy.CalculateField_management(TWLpoldiss, "CU_id", row[3], "PYTHON3")
    arcpy.AddField_management(TWLpoldiss, "CS_id", "Integer")
    arcpy.CalculateField_management(TWLpoldiss, "CS_id", row[1], "PYTHON3")
    arcpy.AddField_management(TWLpoldiss, "TWL", "DOUBLE")
    arcpy.CalculateField_management(TWLpoldiss, "TWL", row[2], "PYTHON3")
    
    # Remove the layers that will be automatically loaded into the Contents
    layers = aprxMap.listLayers()
    for layer in layers:
        if layer.name == "TWL_"+stretchName:
            aprxMap.removeLayer(layer)
            
    print("Coastal Stretch {0} finished".format(row[1]))
    
print('All Coastal Stretches have been finished.')

In [6]:
# Merge all individual shapefiles into one
feature_classes = [] # Empty list where the filenames to be merged will be
walk = arcpy.da.Walk(workspace, datatype="FeatureClass") # Object to iterate

for dirpath, dirnames, filenames in walk:
    for filename in filenames:
        if "TWL_CS_id_" in filename:
            feature_classes.append(os.path.join(dirpath, filename))

arcpy.Merge_management(feature_classes, workspace+"/FloodingMap")

See an example of the final file
<center><img src="images/example-map-flooding.png" width="500"></center>